## SCIKIT-LEARN

In [37]:
import numpy as np 
import pandas as pd
from sklearn.externals import joblib
from pathlib import Path
import string
#from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.svm import SVC
model_path: Path=Path('./')

In [5]:
data = pd.read_csv("spam.csv",encoding='latin-1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":"class", "v2":"text"})
data.head()

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
def pre_process(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

In [11]:
features = data['text'].copy()
features = features.apply(pre_process)

vectorizer = TfidfVectorizer("english")
_features = vectorizer.fit_transform(features)
with open('vectorizer.pkl', 'wb') as vect:
    pickle.dump(vectorizer, vect)


In [12]:
vectorizer = joblib.load(model_path.joinpath('vectorizer.pkl'))

In [13]:
train_x, test_x, train_y, test_y = train_test_split(_features, data['class'], test_size=0.3, random_state=0)

In [14]:
svc = SVC(kernel='sigmoid', gamma=1.0, probability=True)
svc.fit(train_x,train_y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1.0, kernel='sigmoid',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [15]:
# save the model to disk
filename = 'sk_model.pkl'
pickle.dump(svc, open(filename, 'wb'))
clf = joblib.load(model_path.joinpath(filename))

In [17]:
prediction = clf.predict(test_x)
accuracy_score(test_y,prediction)

0.9730861244019139

In [18]:
message = np.array(['click here to win the price'])


In [20]:
data = vectorizer.transform(message).todense()

In [21]:
probas = clf.predict_proba(data)

In [22]:
clf.classes_

array(['ham', 'spam'], dtype=object)

In [23]:
probas

array([[0.02511554, 0.97488446]])

## KERAS

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import pickle
from sklearn.externals import joblib

from keras.engine.saving import model_from_json
from keras.layers import (
    Bidirectional,
    concatenate,
    Dense,
    Embedding,
    LSTM,
    Masking,
    Reshape,
    SpatialDropout1D,
    TimeDistributed,
)

In [32]:
data = pd.read_csv("spam.csv",encoding='latin-1')
data.head()
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":"class", "v2":"text"})
data.head()

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [27]:
X = data.text
Y = data['class']
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [35]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)
max_words = 1000
max_len = 150
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

with open('tokenizer.pkl', 'wb') as tok:
    pickle.dump(tokenizer, tok)

In [ ]:
sequences = tokenizer.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [33]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

In [ ]:
#serialize architecture to JSON
model_json = model.to_json()
with open("architecture.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("weights.h5")
print("Saved model to disk")

In [ ]:
tokenizer = joblib.load('tokenizer.pkl')

# load json and create model
json_file = open('architecture.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("weights.h5")

txt= 'please click this link to win the price'
seq= tokenizer.texts_to_sequences([txt])
padded = sequence.pad_sequences(seq, maxlen=max_len)
pred = model.predict(padded)
print(pred)